In [1]:
#importing a random module to shuffle cards
import random
#declaring variables to store suits, ranks and values of cards in blackjack
suits = ('Hearts', 'Diamonds', 'Spades', 'Clubs')
ranks = ('Two', 'Three', 'Four', 'Five', 'Six', 'Seven', 'Eight', 'Nine', 'Ten', 'Jack', 'Queen', 'King', 'Ace')
values = {'Two':2, 'Three':3, 'Four':4, 'Five':5, 'Six':6, 'Seven':7, 'Eight':8, 'Nine':9, 'Ten':10, 'Jack':10,
             'Queen':10, 'King':10, 'Ace':11}
#declaring a boolean to keep track of game
playing = True

In [2]:
#creating a Card class
class Card:
    
    def __init__(self,suit,rank):
        self.suit = suit
        self.rank = rank
    #defining a str method to print cards
    def __str__(self):
        return(self.rank + " of " + self.suit)

In [3]:
#creating a Deck class and storing all cards in a list
class Deck:
    
    def __init__(self):
        self.deck = []  
        for suit in suits:
            for rank in ranks:
                self.deck.append(Card(suit,rank))
    #defining a str method to print whole deck (debuging purpose) 
    def __str__(self):
        whole_deck = ""
        for card in self.deck:
            whole_deck += '\n'+card.__str__()
        return ("The deck has: " + whole_deck)
    #method to shuffle the deck        
    def shuffle(self):
        random.shuffle(self.deck)
    #method to deal a card (take one card from the deck)        
    def deal(self):
        single_card = self.deck.pop()
        return single_card

In [4]:
#creating a Hand class 
class Hand:
    def __init__(self):
        self.cards = [] #creating a empty list to store cards in posession of player and dealer 
        self.value = 0  #creating a value attribute to keep track of current value in hand 
        self.aces = 0   #creating aces attribute to keep track of number of aces in hand (as their value behaves differently than other cards)
    #method to add cards value and count if the card is a ace
    def add_card(self,card):
        self.cards.append(card)
        self.value += values[card.rank]
        
        if card.rank == 'Ace':
            self.aces += 1
        
    #method that treats ace as a value of 11 if the total value including the ace is less than 21 and as 1 if the value is more than 21
    def adjust_for_ace(self):
        while self.value >21 and self.aces:
            self.value -= 10
            self.aces -= 1

In [7]:
#creating a Chips class 
class Chips:
    #creating attributes to keep track of total chips and a bet
    def __init__(self,total):
        self.total = total  
        self.bet = 0
        
    def win_bet(self):
        self.total += self.bet
    
    def lose_bet(self):
        self.total -= self.bet
        

In [10]:
#creating a function to ask how many chips does user want to bet
def take_bet(chips):
    while True:
        try:
            chips.bet = int(input("How many credits would you like to bet?"))
        except:
            print("How many credits would you like to bet?")
        else:
            if chips.bet<=0:
                print("You have to bet more than 0 credtis")
            elif chips.bet > chips.total:
                print('Sorry you dont have enough chips. You have'.format(chips.total))
                
            else:
                break

In [11]:
#function that uses previously defined methods aswell as  deck and hand classes to take a card
def hit(deck,hand):
    
    single_card = deck.deal()
    hand.add_card(single_card)
    hand.adjust_for_ace()

In [12]:
#function that uses deck and hand classes and asks for a players move
def hit_or_stand(deck,hand):
    global playing  
    
    while True:
        hit_or_stand = input("Would you like to Hit or Stand?")
        
        if hit_or_stand == 'hit':
            hit(deck,hand)  

        elif hit_or_stand == 'stand':
            print("Player stands. Dealer is playing.")
            playing = False

        else:
            print("Sorry, please try again.")
            continue
        break

In [16]:
#function to display cards
def show_some(player,dealer): #display all player cards and cover dealers first card
    print("\nDealer's Hand:")
    print(" <card hidden>")
    print('',dealer.cards[1])  
    print("\nPlayer's Hand:", *player.cards, sep='\n ')
    
def show_all(player,dealer):#display all cards (to use at the end of the game)
    print("\nDealer's Hand:", *dealer.cards, sep='\n ')
    print("Dealer's Hand =",dealer.value)
    print("\nPlayer's Hand:", *player.cards, sep='\n ')
    print("Player's Hand =",player.value)

In [15]:
#if player busts
def player_busts(player,dealer,chips):
        print("You have busted!")
        chips.lose_bet()
#if player wins by higher value than dealer        
def player_wins(player,dealer,chips):
        print("You have won!")
        chips.win_bet()
#if dealer busts
def dealer_busts(player,dealer,chips):
        print("Player won, dealer busted!")
        chips.win_bet()
#if dealer wins by higher valye than player
def dealer_wins(player,dealer,chips):
        print("Dealer won!")
        chips.lose_bet()
def push(player,dealer):
    print("Thats a draw!")

In [17]:
#GAME

#setting up a chips class for 500 credits
player_chips = Chips(500)

while True:
   
    print("Welcome to blackjack!")
    
    #creating a deck and shuffling the cards 
    deck = Deck()
    deck.shuffle()
    
    #setting up a hand class for player and dealer
    player_hand = Hand()
    dealer_hand = Hand()
    
    #giving two cards to a player
    single_card = deck.deal()
    player_hand.add_card(single_card)
    single_card = deck.deal()
    player_hand.add_card(single_card)
    
    #giving two cards to a dealer
    single_card = deck.deal()
    dealer_hand.add_card(single_card)
    single_card = deck.deal()
    dealer_hand.add_card(single_card)
    
    #asking for players bet
    take_bet(player_chips)
    
    #showing cards
    show_some(player_hand,dealer_hand)
    
    #staring the game
    playing = True
    while playing == True:  
        
        #asking a player for their next move
        hit_or_stand(deck,player_hand)
        
        #showing cards
        show_some(player_hand,dealer_hand)
        
        # if player busts end the game
        if player_hand.value > 21:
            player_busts(player_hand,dealer_hand,player_chips)
            break

    #if player hasn't busted, play Dealer's hand until Dealer reaches 17
    if player_hand.value <=21:
        
        while dealer_hand.value < 17:
            hit(deck,dealer_hand)


        #end of the game

        #show all cards
        show_all(player_hand,dealer_hand)
            
        #calculate and present the winner
        if dealer_hand.value > 21:
            dealer_busts(player_hand,dealer_hand,player_chips)

        elif dealer_hand.value > player_hand.value:
            dealer_wins(player_hand,dealer_hand,player_chips)

        elif dealer_hand.value < player_hand.value:
            player_wins(player_hand,dealer_hand,player_chips)

        else:
            push(player_hand,dealer_hand)        

    #informing the player of their total chips left
    print("You have {} chips left".format(player_chips.total))
    
    #asking a player to play again
    again = input("Would you like to play another hand? Enter 'yes' or 'no' ")
    if again == 'yes':
        playing=True
        continue
    else:
        print("Thanks for playing!")
        break


Welcome to blackjack!
How many credits would you like to bet?100

Dealer's Hand:
 <card hidden>
 Four of Hearts

Player's Hand:
 Six of Clubs
 Six of Spades
Would you like to Hit or Stand?hit

Dealer's Hand:
 <card hidden>
 Four of Hearts

Player's Hand:
 Six of Clubs
 Six of Spades
 King of Diamonds
You have busted!
You have 400 chips left
Would you like to play another hand? Enter 'yes' or 'no' no
Thanks for playing!
